In [1]:
import numpy as np
import scipy.linalg as sla
import numpy.linalg as la
from scipy.special import expit as sigmoid
import scipy as sc
from statsmodels.tsa.api import VAR

In [2]:
varables = 4
observations = 1000
window = 5
varTime = varables*window
#to use later 
times = 10

In [3]:
def qDag(w):
    return np.trace(sc.linalg.expm(np.multiply(w,w))) - w.shape[0]
def get_r(off):
    return lambda n: np.random.rand(n)+off

In [4]:
dagY = sc.sparse.random(varables, varables*window, density=0.15, data_rvs=get_r(2)).A

dagX = np.eye(varables, varables)
while qDag(dagX) != 0:
    dagX = sc.sparse.random(varables, varables, density=0.45, data_rvs=get_r(2)).A
    np.fill_diagonal(dagX, 0)

In [5]:
print(qDag(dagX))
dagX

0.0


array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 2.26162701, 2.38027911],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 2.35197219, 0.        ]])

In [6]:
def rand(n):
    return 2*(np.random.rand(n)-.5)

Y_pre = rand(observations*varables*window).reshape(varables*window, observations)
print(dagY.shape)
print(Y_pre.shape)

(4, 20)
(20, 1000)


In [7]:
#X = XW + YA + Z

def randFlat(X):
    return rand(X.shape[0]*X.shape[1]).reshape(*X.shape)

def update(preY, preX, dagY, dagX, rand):
    Xy = np.matmul(dagY, preY)
    Xr = rand(preX)
    Xx = np.matmul(dagX, Xr)
    X = Xr + Xx + Xy
    
    return X

def step(preY, dagY, dagX, window, var, rand):
    preX = preY[:var]
    preY = preY[:window*var]
    return update(preY, preX, dagY, dagX, rand)

def makeSynthData(preY, times, obs, var, pad, rand=rand):
    window = preY.shape[0]//var
    
    #for i in range(pad+window):
     #   newX = step(preY, dagY, dagX, window, var, rand)
      #  preY = np.append(newX, preY, axis=0)
        
    Y = Y_pre[:window*var]

    for i in range(times - window):
        newX = step(Y, dagY, dagX, window, var, rand)
        Y = np.append(newX, Y, axis=0)
        
    return Y
    



#later
#W = np.random.rand(varables*observations).reshape(varables,observations)

In [8]:
XY = makeSynthData(Y_pre, 6, observations, varables, 0, randFlat)


In [9]:
Y=XY[varables:window*varables+varables]
X=XY[:varables]

In [10]:
X.shape

(4, 1000)

In [11]:
C_init = np.random.rand(varables*varables*window + varables*varables)

In [12]:
def f_DYNOTEARS(X, Y, lA, lB):
    def aux(C):
        a = C[:varables*window*varables].reshape(varables, varables*window)
        w = C[varables*window*varables:].reshape(varables, varables)
    
        loss = np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/X.shape[1]
        #print()
        #print(loss)
        #print(np.square(np.linalg.norm(X)))
        #print(np.square(np.linalg.norm(X - np.matmul(w, X))))
        #print(np.square(np.linalg.norm(X - np.matmul(a, Y))))
        #print(np.linalg.norm(w.flatten(), 1))
        #print(np.linalg.norm(a.flatten(), 1))
    
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

In [13]:
def constr_DYNOTEARS(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS(X,Y,.01,.1) , C_init, constraints=cons).x

In [14]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [15]:
A[A<.3] = 0
A.round()

array([[0., 2., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 2.],
       [0., 1., 1., 0., 0., 0., 0., 2., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 1.],
       [0., 3., 3., 0., 0., 0., 0., 0., 2., 2., 0., 2., 0., 0., 0., 0.,
        0., 0., 0., 3.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0.,
        3., 0., 0., 0.]])

In [16]:
dagY.round()

array([[0., 2., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 2.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 3., 3., 0., 0., 0., 0., 0., 2., 2., 0., 2., 0., 0., 0., 0.,
        0., 0., 0., 3.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0.,
        3., 0., 0., 0.]])

In [17]:
W.round(1)

array([[ 0. ,  0. ,  0. , -0. ],
       [ 0. ,  0. , -0.2,  1.2],
       [-0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  2. ,  0. ]])

In [18]:
dagX.round(1)

array([[0. , 0. , 0. , 0. ],
       [0. , 0. , 2.3, 2.4],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 2.4, 0. ]])

In [19]:
dagX.T.round(1)

array([[0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 2.3, 0. , 2.4],
       [0. , 2.4, 0. , 0. ]])

In [40]:
def H1ldet(W):
    det = np.linalg.det(np.eye(W.shape[0]) - np.multiply(W,W))
    if (det <= 0):
        print(det)
    return -np.log(det)

def constr_DAGMA(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return H1ldet(w)

A_init = np.random.rand(varables*varables*window)

W_init = np.random.rand(varables*varables).reshape(varables, varables)
np.fill_diagonal(W_init, 0)
while H1ldet(W_init) is np.nan:
    W_init = np.random.rand(varables*varables).reshape(varables, varables)
    np.fill_diagonal(W, 0)

C_init = np.append(A_init, W_init.reshape(varables*varables), axis = 0)

cons = {'type':'eq', 'fun': constr_DAGMA}

C = sc.optimize.minimize(f_DYNOTEARS(X,Y,.01,.1) , C_init, constraints=cons).x

-52262324.70803839
-1.1517328713312862
-5505127.120653363
-0.0027023861038550706
-5.835858683559525e+17
-5982204785.973656
-35.73119606058
-527.540113059798
-52630818.82163241
-0.032226508306203945
-18.461243780152547
-76.9355377099625
-4.03146807149412
-1.434695478566765
-0.5369413652753453
-1.9364993831293325


/var/folders/z_/p0tz320d6j13q0rkv_f35brm0000gn/T/ipykernel_5806/3174799670.py:5: RuntimeWarning: invalid value encountered in log
  return -np.log(det)


In [41]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [42]:
A[A<.3] = 0
A.round()

array([[0., 2., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 2.],
       [0., 1., 1., 0., 0., 0., 0., 2., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 1.],
       [0., 3., 3., 0., 0., 0., 0., 0., 2., 2., 0., 2., 0., 0., 0., 0.,
        0., 0., 0., 3.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0.,
        3., 0., 0., 0.]])

In [43]:
dagY.round()

array([[[3., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 3.]],

       [[3., 3., 2., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0.],
        [2., 0., 0., 3., 0.]],

       [[0., 0., 0., 0., 0.],
        [2., 2., 0., 0., 0.],
        [0., 2., 0., 2., 0.],
        [0., 3., 0., 2., 3.]],

       [[0., 2., 0., 0., 3.],
        [0., 0., 0., 2., 0.],
        [3., 0., 0., 0., 0.],
        [0., 0., 0., 3., 0.]]])

In [44]:
W.round(1)

array([[ 0. ,  0. ,  0. , -0. ],
       [ 0. ,  0. , -0.2,  1.2],
       [ 0. ,  0. ,  0. ,  0. ],
       [-0. ,  0. ,  2. ,  0. ]])

In [25]:
dagX.round(1)

array([[0. , 0. , 0. , 0. ],
       [0. , 0. , 2.3, 2.4],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 2.4, 0. ]])

In [26]:
dagX.T.round(1)

array([[0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 2.3, 0. , 2.4],
       [0. , 2.4, 0. , 0. ]])

In [27]:
C_init = np.random.rand(varables*varables*window + varables*varables)
H1ldet(C_init.T)

-34.420498789207095


/var/folders/z_/p0tz320d6j13q0rkv_f35brm0000gn/T/ipykernel_5806/726157458.py:5: RuntimeWarning: invalid value encountered in log
  return -np.log(det)


nan

In [28]:
def metrics(A,dagY, W, dagX):
    A = np.absolute(A) > 0.3
    dagY = np.absolute(dagY) > .5
    W = np.absolute(W) > 0.3
    dagX = np.absolute(dagX) > .5
    
    preWrong = A != dagY
    wWrong = W != dagX
    #assumes cant be both wrong and inv
    wInverse = W.T & dagX
    
    if ((W + W.T)==2).any():
        print(W)
        print(W.T)
        raise "error"
    
    #print(A)
    #print(dagY)
    #print(W)
    #print(dagX)
    #print()
    #print(preWrong)
    #print(wWrong)
    #print(wInverse)
    
    return {
        "preWrong": sum(sum(preWrong)),
        "wWrong": sum(sum(wWrong)),
        "wInverse": sum(sum(wInverse))
    }
    

In [29]:
metrics(A,dagY, W, dagX)

{'preWrong': 10, 'wWrong': 2, 'wInverse': 0}

In [30]:
A.round()

array([[0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0.]])

In [31]:
dagY.round()

array([[0., 2., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 2.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 3., 3., 0., 0., 0., 0., 0., 2., 2., 0., 2., 0., 0., 0., 0.,
        0., 0., 0., 3.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0.,
        3., 0., 0., 0.]])

In [32]:
np.matmul(np.eye(varables, varables) - W, X).round(1)

array([[ 2.4, -3.1, -2.6, ...,  0.8,  1.8, -0.2],
       [ 3.4, -1.6, -0.8, ...,  2.8, -2.2, -1.1],
       [ 2.7,  2.9, -2.4, ...,  4. ,  0.4,  0.8],
       [ 2.7,  1.1, -0.5, ...,  3.5,  0. , -2.2]])

In [33]:
#tensor representation for timeseries

In [34]:
dagY = sc.sparse.random(varables, varables*window, density=0.25, data_rvs=get_r(2)).A.reshape(varables, varables, window)

dagX = np.eye(varables, varables)
while qDag(dagX) != 0:
    dagX = sc.sparse.random(varables, varables, density=0.25, data_rvs=get_r(2)).A
    np.fill_diagonal(dagX, 0)

In [35]:
dagY.round().shape

(4, 4, 5)

In [36]:
#tensor representation 